In [ ]:
from fastai.vision.all import *
from tqdm.notebook import  tqdm

PATH = Path('../input/optiver-realized-volatility-prediction')

**UPDATE Aug 11th**: I've cleaned up and simplified the model, now all it uses are 1D convolution instead the 5x1 2D convs. Also I experimented with numbers of layers and channels and managed to improve the score slightly. 
Note that still it only uses a single fold (80% of book data) and no trade data at all, so there's rooom for improvement.

# Solution overview

### This notebook demonstrates an approach where a neural network is trained on the raw book data. I'm not adding any engineered features, so the network starts with no concept of prices, returns, volatility or logarithms.

### Each input sample is simply a 600x8 tensor representing the 8 numerical columns of the book data at each second of the 10 minute window.

## The model
I'm using a convolutional neural network with architecture inspired by ResNet. With a total of 65 1D convolutional layers, followed by a single dense layer.

With a small number of channels and 5x1 convolutions this is still fairly lightweight and doesn't take long to infere. Training took around 25 minutes on a single RTX3090 GPU.

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv1d(ch, ch, kernel_size = 5, padding = 2, padding_mode='replicate'),
            nn.BatchNorm1d(ch),
            nn.ReLU(),
            nn.Conv1d(ch, ch, kernel_size = 5, padding = 2, padding_mode='replicate'),
            nn.BatchNorm1d(ch),
        )
        
    def forward(self, x):
        res = self.layers(x) + x
        res = F.relu(res)
        return res

class ResnetModel(nn.Module):
    def __init__(self):
        super().__init__()
        chan = 32
        layers = [nn.Conv1d(8, chan, kernel_size=1)]
        for _ in range(8):
            layers += [ResBlock(chan), ResBlock(chan), ResBlock(chan), ResBlock(chan)
                       , nn.AvgPool1d(2, padding=1),
                      ]
        layers += [Flatten(), nn.Dropout()]   
        self.conv_layers = nn.Sequential(*layers)
        self.classifier = nn.Linear(chan*4, 1)
        
    def forward(self, x):
        feat = self.conv_layers(x)
        res = self.classifier(feat)
        return sigmoid_range(res, 0, .1).view(-1)

In [ ]:
data_dir = PATH/'book_test.parquet'
model_file = '../input/optiver-resnet-model/resnet_model2.pth'
model = torch.load(model_file)

### Stats from the train data used for normalization:

In [ ]:
means = tensor([  0.9997,   1.0003, 769.9902, 766.7346,   0.9995,   1.0005, 959.3417,
        928.2203])
stds = tensor([3.6881e-03, 3.6871e-03, 5.3541e+03, 4.9549e+03, 3.7009e-03, 3.6991e-03,
        6.6838e+03, 5.7353e+03])

### See the discussion [here](https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/251775)

In [ ]:
def fix_offsets(data_df):
    offsets = data_df.groupby(['time_id']).agg({'seconds_in_bucket':'min'})
    offsets.columns = ['offset']
    data_df = data_df.join(offsets, on='time_id')
    data_df.seconds_in_bucket = data_df.seconds_in_bucket - data_df.offset
    return data_df

### Explained [here](https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/251277)

In [ ]:
def ffill(data_df):
    data_df=data_df.set_index(['time_id', 'seconds_in_bucket'])
    data_df = data_df.reindex(pd.MultiIndex.from_product([data_df.index.levels[0], np.arange(0,600)], names = ['time_id', 'seconds_in_bucket']), method='ffill')
    return data_df.reset_index()

In [ ]:
def load_data(fname):
    data = pd.read_parquet(fname)
    stock_id = str(fname).split('=')[1]
    time_ids = data.time_id.unique()
    row_ids = list(map(lambda x:f'{stock_id}-{x}', time_ids))
    data = fix_offsets(data)
    data = ffill(data)
    data = data[['bid_price1', 'ask_price1', 'bid_size1', 'ask_size1','bid_price2', 'ask_price2', 'bid_size2', 'ask_size2']].to_numpy()
    data = torch.tensor(data.astype('float32'))
    data = (data - means) / stds
    return data, row_ids

In [ ]:
def get_preds(data, model):
    data = data.view(-1,600,8).permute(0, 2, 1)
    with torch.no_grad():
        preds = model(data.cuda())

    return preds

In [ ]:
%%time
all_preds = []
for fname in tqdm(data_dir.ls()):
    data, row_ids = load_data(fname)
    preds = get_preds(data, model)
    df_pred = pd.DataFrame(zip(row_ids, preds.tolist()),columns=['row_id', 'target'])
    all_preds.append(df_pred)

In [ ]:
df_pred = pd.concat(all_preds)
df_pred.to_csv('submission.csv', index=False)

In [ ]:
pd.read_csv('submission.csv').head()